In [65]:
import pandas as pd
import requests
import json
import time
import random
import os
from datetime import datetime
import comfunc as cf


def urltjason(url):
    user_agent = {'User-agent': 'Chrome/107.0.0.0'}
    r = requests.get(url,  headers=user_agent)
    print(r, '狀態回應')
    list_of_dicts = r.json()
    # print (list_of_dicts)
    return (list_of_dicts)


def getstockdata(date, stock_no):
    html = urltjason(
        'https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=%s&stockNo=%s' % (date, stock_no))
    stock_data = html['data']
    col_name = html['fields']
    global stockname
    stockname = html['title']
    return pd.DataFrame(data=stock_data, columns=col_name)


def historydata(date, enddate, stockdata, stock_no):
    for i in range(1, 5):  # 累積<5年資料
        for j in range(1, 13):
            print(date)
            stockdata = stockdata.append(getstockdata(date, stock_no))
            time.sleep(random.uniform(3, 6))
            date = date + 100
            if date > enddate:
                break
        date = date + 10000 - 1200
        if date > enddate:
            break
    return stockdata


def makehistorylist(historylistdir):
    # 建立歷史資料檔名列表historylist
    historylistdir = historylistdir
    historylist = pd.DataFrame(os.listdir(historylistdir))
    print(historylist)
    historylist.to_csv("historylist.csv", index=False, encoding='utf-8-sig')


def lowshadow(pf, twstrdate, stockname):
    ShadowAlert = tuple()
    try:
        if pf.at[twstrdate, "開盤價"] > pf.at[twstrdate, "收盤價"]:
            loshadow = (pf.at[twstrdate, "最高價"] -
                        pf.at[twstrdate, "收盤價"]) / pf.at[twstrdate, "開盤價"]
            # print('綠下影線=', loshadow)
        else:
            loshadow = (pf.at[twstrdate, "開盤價"] -
                        pf.at[twstrdate, "最低價"]) / pf.at[twstrdate, "開盤價"]
            # print('紅下影線=', loshadow)
        if loshadow > 0.03:
            print(stockname, '有長下影線', round(loshadow, 2))
            ShadowAlert = '有長下影線=', round(loshadow, 2)
    except:
        print(stockname, 'shadow pass')
        pass
    return ShadowAlert


# 要更新的月份
# date = 20230101
path = os.getcwd()
# 歷史檔案清單與清單內股票代號
hislist = pd.read_csv(path+'/historylist.csv', encoding='utf-8-sig')
stock_nolist = []

In [66]:
for i in range(0, len(hislist)):  # len(hislist)
    hislists = hislist.iat[i, 0].split()
    # print (hislists,hislists[1])
    stock_nolist.append(hislists[1])
stock_nolist = pd.DataFrame(stock_nolist)

for i in range(0, 1):  # len(hislist)

    # 取得自有歷史資料
    thehistory = pd.read_csv(path+'/112kdnewhistory/' +
                             hislist.iat[i, 0])
    print(thehistory)

            日期        成交股數         成交金額    開盤價    最高價    最低價    收盤價   漲跌價差  \
0    111/03/01  14071999.0  671571062.0  47.80  47.85  47.50  47.80  +0.45   
1    111/03/02   8187419.0  391264267.0  47.80  48.00  47.60  48.00  +0.20   
2    111/03/03   7565742.0  362783289.0  47.95  48.05  47.80  48.05  +0.05   
3    111/03/04  11071393.0  527906917.0  47.80  47.85  47.50  47.65  -0.40   
4    111/03/07  16991368.0  799794535.0  47.45  47.45  46.90  47.10  -0.55   
..         ...         ...          ...    ...    ...    ...    ...    ...   
235  112/02/15  19116424.0  719132693.0  37.35  37.85  37.30  37.55    0.3   
236  112/02/17  18234212.0  688163008.0  37.40  38.00  37.25  38.00    0.5   
237  112/02/20  16664374.0  634546656.0  37.95  38.40  37.80  38.25   0.25   
238  112/02/21   9443068.0  361950570.0  38.40  38.50  38.15  38.35    0.1   
239  112/02/22  19617132.0  745735095.0  38.15  38.30  37.85  38.05   -0.3   

        成交筆數 filter       rsv         k         d  
0     7420.

In [27]:
    print()

37.465


In [67]:
    print (thehistory.tail())

    new=thehistory
    print (new.tail())

            日期        成交股數         成交金額    開盤價    最高價    最低價    收盤價  漲跌價差  \
235  112/02/15  19116424.0  719132693.0  37.35  37.85  37.30  37.55   0.3   
236  112/02/17  18234212.0  688163008.0  37.40  38.00  37.25  38.00   0.5   
237  112/02/20  16664374.0  634546656.0  37.95  38.40  37.80  38.25  0.25   
238  112/02/21   9443068.0  361950570.0  38.40  38.50  38.15  38.35   0.1   
239  112/02/22  19617132.0  745735095.0  38.15  38.30  37.85  38.05  -0.3   

        成交筆數 filter       rsv         k         d  
235  11069.0    NaN  0.837838  0.752362  0.723870  
236   9608.0    NaN  1.000000  0.834908  0.779877  
237  10140.0    NaN  0.931818  0.867211  0.845676  
238   6552.0    NaN  0.926829  0.887084  0.873835  
239  10712.0    NaN  0.780488  0.851552  0.875240  
            日期        成交股數         成交金額    開盤價    最高價    最低價    收盤價  漲跌價差  \
235  112/02/15  19116424.0  719132693.0  37.35  37.85  37.30  37.55   0.3   
236  112/02/17  18234212.0  688163008.0  37.40  38.00  37.25  38.00   0

In [82]:
    twentyMA=thehistory['收盤價'].tail(20).mean()
    sixtyMA=thehistory['收盤價'].tail(60).mean()
    new.at[len(new)-1,'20ma']=twentyMA
    print (new.loc[-1:])
    #print (new.tail().values[-1])

            日期        成交股數         成交金額    開盤價    最高價    最低價    收盤價   漲跌價差  \
0    111/03/01  14071999.0  671571062.0  47.80  47.85  47.50  47.80  +0.45   
1    111/03/02   8187419.0  391264267.0  47.80  48.00  47.60  48.00  +0.20   
2    111/03/03   7565742.0  362783289.0  47.95  48.05  47.80  48.05  +0.05   
3    111/03/04  11071393.0  527906917.0  47.80  47.85  47.50  47.65  -0.40   
4    111/03/07  16991368.0  799794535.0  47.45  47.45  46.90  47.10  -0.55   
..         ...         ...          ...    ...    ...    ...    ...    ...   
237  112/02/20  16664374.0  634546656.0  37.95  38.40  37.80  38.25   0.25   
238  112/02/21   9443068.0  361950570.0  38.40  38.50  38.15  38.35    0.1   
239  112/02/22  19617132.0  745735095.0  38.15  38.30  37.85  38.05   -0.3   
240        NaN         NaN          NaN    NaN    NaN    NaN    NaN    NaN   
241        NaN         NaN          NaN    NaN    NaN    NaN    NaN    NaN   

        成交筆數 filter       rsv         k         d       20ma  


In [78]:
new[new.columns[-1]]


0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
         ...   
235    0.723870
236    0.779877
237    0.845676
238    0.873835
239    0.875240
Name: d, Length: 240, dtype: float64